### **LCEL Chain 메모리 추가**

대화 내용 기억

임의의 체인에 메모리를 추가하는 방법

In [44]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_groq import ChatGroq
import warnings
warnings.filterwarnings("ignore")

# 모델 정의
llm = ChatGroq(model = 'gemma2-9b-it')

# 프롬프트 생성
prompt = ChatPromptTemplate.from_messages(
    [   
        ('system', 'You are a helpful AI assitants'),
        MessagesPlaceholder(variable_name='chat_history'),
        ('user', '{input}')
    ]
)

In [45]:
# 대화 버퍼 메모리 생성, 메세지 반환 기능 활성화
memory = ConversationBufferMemory(return_messages=True, memory_key = 'chat_history')  # 대화 기억을 저장하기 위한 메모리리
print(memory.load_memory_variables({})) 

{'chat_history': []}


`RunnablePassthrough.assign` 사용

`chat_history` 변수에 `memory.load_memory_variables` 함수의 결과를 할당하고, 이 결과에서 `chat_history` 키에 해당하는 값을 추출.

In [46]:
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables)  # memory.load_memory_variables를 실행시키기 위한 RunnableLambda
    | itemgetter("chat_history")  # 딕셔너리의 키값을 가져오기 위한 itemgetter(chat_history가 딕셔너리 형태기 때문에에)
)

In [34]:
# itemgeter 안쓸때 
runnable.invoke({'input' : '안녕하세요'})   # runnable.assign은 반드시 딕셔너리로 입력해야함

{'input': '안녕하세요', 'chat_history': {'chat_history': []}}

In [36]:
# itemgeter 쓸때
runnable.invoke({'input' : '안녕하세요'})  # 'chat_history' : []  => 딕셔너리에서 키값을 가져와 리스트만 출력됨

{'input': '안녕하세요', 'chat_history': []}

In [51]:
# Chain 생성
chain = runnable | prompt | llm

# 응답 생성
response = chain.invoke({'input' : "안녕하세요 직장이고 대학원을 다니고 있어요"})
print(response.content)  # 생성된 응답을 출력합니다.

안녕하세요! 직장과 대학원까지 병행하시는군요. 정말 멋지고 힘든 일이겠네요.  

어떤 분야를 공부하고 계신가요? 혹시 힘든 일이 있으신가요?  

조금이라도 도움이 될 수 있으면 좋겠어요. 😊 💪



In [52]:
# 아직 저장 안됨
memory.load_memory_variables({}) 

{'chat_history': [HumanMessage(content='안녕하세요', additional_kwargs={}, response_metadata={}),
  AIMessage(content='안녕하세요! 👋  \n\n무엇을 도와드릴까요? 😊  \n\n저는 다양한 질문에 답하고, 텍스트를 생성하고, 번역을 도와드릴 수 있습니다.  \n\n궁금한 것이 있으시면 언제든지 물어보세요! 😊\n', additional_kwargs={}, response_metadata={})]}

In [53]:
# 입력 데이터와 응답을 메모리에 저장 
memory.save_context(
    {"user": "안녕하세요 직장이고 대학원을 다니고 있어요"}, {"ai": response.content}
)


In [59]:
# 메모리 저장기록 출력
memory.load_memory_variables({})['chat_history']

[HumanMessage(content='안녕하세요', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요! 👋  \n\n무엇을 도와드릴까요? 😊  \n\n저는 다양한 질문에 답하고, 텍스트를 생성하고, 번역을 도와드릴 수 있습니다.  \n\n궁금한 것이 있으시면 언제든지 물어보세요! 😊\n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='안녕하세요 직장이고 대학원을 다니고 있어요', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요! 직장과 대학원까지 병행하시는군요. 정말 멋지고 힘든 일이겠네요.  \n\n어떤 분야를 공부하고 계신가요? 혹시 힘든 일이 있으신가요?  \n\n조금이라도 도움이 될 수 있으면 좋겠어요. 😊 💪\n', additional_kwargs={}, response_metadata={})]

In [61]:
# 기억하고 있는지 확인
response = chain.invoke({"input": "제가 어떤걸 하고 있다고 말했는지 기억해?"})
# 답변을 출력합니다.
print(response.content)

네, 기억해요! 직장과 대학원을 병행하고 계신다고 말씀하셨어요.  정말 멋지네요!  어떤 분야를 공부하고 계신가요? 😊






### **커스텀 ConversationChain 구현 예시**

In [ ]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, Runnable
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# ChatOpenAI 모델을 초기화합니다.
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# 대화형 프롬프트를 생성합니다. 이 프롬프트는 시스템 메시지, 이전 대화 내역, 그리고 사용자 입력을 포함합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

# 대화 버퍼 메모리를 생성하고, 메시지 반환 기능을 활성화합니다.
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

class MyConversationChain(Runnable):

    def __init__(self, llm, prompt, memory, input_key="input"):

        self.prompt = prompt
        self.memory = memory
        self.input_key = input_key

        self.chain = (
            RunnablePassthrough.assign(
                chat_history=RunnableLambda(self.memory.load_memory_variables)
                | itemgetter(memory.memory_key)  # memory_key 와 동일하게 입력합니다.
            )
            | prompt
            | llm
            | StrOutputParser()
        )

    def invoke(self, query, configs=None, **kwargs):
        answer = self.chain.invoke({self.input_key: query})
        self.memory.save_context(inputs={"human": query}, outputs={"ai": answer})
        return answer

conversation_chain = MyConversationChain(llm, prompt, memory)

In [9]:
conversation_chain.invoke("안녕하세요? 만나서 반갑습니다. 제 이름은 테디 입니다.")

'안녕하세요, 테디님! 만나서 반갑습니다. 어떻게 도와드릴까요?'

In [74]:
conversation_chain.invoke("제 이름이 뭐라고요?")

'당신의 이름은 테디입니다! 맞나요?'

In [75]:
conversation_chain.invoke("앞으로는 영어로만 답변해주세요 알겠어요?")

"Got it! I'll respond in English from now on. How can I assist you today?"

In [76]:
conversation_chain.invoke("제 이름을 다시 한 번 말해주세요")

'Your name is Teddy.'

In [77]:
conversation_chain.memory.load_memory_variables({})["chat_history"]

[HumanMessage(content='안녕하세요? 만나서 반갑습니다. 제 이름은 테디 입니다.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요, 테디님! 만나서 반갑습니다. 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='제 이름이 뭐라고요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='당신의 이름은 테디입니다! 맞나요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='앞으로는 영어로만 답변해주세요 알겠어요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Got it! I'll respond in English from now on. How can I assist you today?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='제 이름을 다시 한 번 말해주세요', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Your name is Teddy.', additional_kwargs={}, response_metadata={})]